# **Ekploracyjna Analiza Danych (EDA)**

### **Cel Analizy**
1.  **Dystrybucja ocen ekspertów:** Analiza poziomu zbalansowania etykiet w zbiorze danych.
1.  **Dystrybucja długości raportów:** Analiza ilości znaków per dokument.

#### **Import Bibliotek**

In [ ]:
import pandas as pd
import plotly.express as px

In [ ]:
with open('data/data.jsonl', 'r') as f:
    df = pd.read_json(f, lines=True)

#### **Kryteria (etykiety)**
* ##### **Kryterium 1** - Plan Transformacji Klimatycznej (ESRS E1-1)
* ##### **Kryterium 2** - Identyfikacja i Zarządzanie Ryzykami (ESRS E1-2)
* ##### **Kryterium 4** - Definicja Granic Konsolidacji (ESRS E1-6 Metodyka)
* ##### **Kryterium 6** - Raportowanie Danych Historycznych (ESRS 1 + ESRS E1)
* ##### **Kryterium 7** - Ujawnienie Wskaźników Intensywności (ESRS E1-6)
* ##### **Kryterium 8** - Wiarygodność Zdefiniowanych Celów (ESRS E1-4)

In [ ]:
criterias = ['Kryterium 1', 'Kryterium 2', 'Kryterium 4', 'Kryterium 6', 'Kryterium 7', 'Kryterium 8']

## **Dystrybucja ocen ekspertów (etykiet)**

In [ ]:
results_list = []

df_len = len(df)
for i in range(len(df['labels'][0])):
  count_ones = df[df['labels'].str[i] == 1]['labels'].count()
  count_zeros = df_len - count_ones
  row_data = {
      "Kryterium": criterias[i],
      "1": count_ones,
      "0": count_zeros
  }

  results_list.append(row_data)

data_labels = pd.DataFrame(results_list)

In [ ]:
fig = px.bar(
  data_labels, 
  x=data_labels['Kryterium'], 
  y=[data_labels['1'], data_labels['0']],
  text_auto=True
)

fig.update_layout(
  xaxis_title="Kryteria",
  yaxis_title="",
  plot_bgcolor='white',
  legend_title_text = 'Etykiety'
)
fig.show()

### **Dystrybucja ocen ekspertów (etykiet) - wnioski:**

Na podstawie przeprowadzonej analizy, można stwierdzić że zbiór jest niezbalansowany pod kątem występowania klas, głównie dla kryteriów 2, 6 i 8, z przewagą klasy negatywnej (0).

Podczas fine-tuningu modelu wymagane będzie ustawienie niestandardowych progów dla ewaluacji wyników.

Uzasadnione również będzie zastosowanie bardziej agresywnych technik optymalizacji funkcji straty np. Focal Loss.

## **Dystrybucja długości raportów**

In [ ]:
results_list = []
for i in range(len(df['text'])):
  results_list.append(len(df['text'][i]))

In [ ]:
fig = px.histogram(
  results_list, 
  x=results_list,
  text_auto=True,
  nbins=50
)

fig.update_layout(
  yaxis_title = "Ilość raportów",
  xaxis_title = "Ilość znaków",
  bargap=0.1
)

fig.show()

### **Dystrybucja długości raportów - wnioski:**

Dystrybucja jest prawostronnie skośna, z większością raportów o długości od 0 do 400 000 znaków. Długi ogon rozkładu potwierdza obecność kilku bardzo obszernych raportów.

Dla procesu treningu modelu oznaczać to będzie konieczność podzielenia raportów na fragmenty oraz zastosowania technik przesuwnego okna podczas tokenizacji. Końcowo będzie to wymagało zastosowania odpowiedniej techniki agregacji wyników.